In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import openai
import pandas as pd
import json

openai.api_key = 'sk-dddfebca99f24567ba0742500e1f3fc0'
openai.api_base = 'https://api.deepseek.com/v1'


In [ ]:
with open('/content/gdrive/MyDrive/CCL/DeepSeek/train_sent_idiom.json', "r", encoding="utf-8") as f:
    sent_idiom = json.load(f)
with open('/content/gdrive/MyDrive/CCL/DeepSeek/train_sent_literal.json', "r", encoding="utf-8") as f:
    sent_literal = json.load(f)
with open('/content/gdrive/MyDrive/CCL/DeepSeek/train_pie_idiom.json', "r", encoding="utf-8") as f:
    pie_idiom = json.load(f)
with open('/content/gdrive/MyDrive/CCL/DeepSeek/train_pie_literal.json', "r", encoding="utf-8") as f:
    pie_literal = json.load(f)
dict_idiom = {k:e for k, e in zip(pie_idiom,sent_idiom)}
dict_literal = {k:e for k, e in zip(pie_literal, sent_literal)}


In [ ]:
with open('/content/gdrive/MyDrive/CCL/GPT/sent_dev.json', "r", encoding="utf-8") as f:
    sent_dev = json.load(f)
with open('/content/gdrive/MyDrive/CCL/GPT/pie_dev.json', "r", encoding="utf-8") as f:
    pie_dev = json.load(f)
dict_dev = {k:e for k, e in zip(pie_dev,sent_dev)}
print(dict_dev)

In [ ]:
with open('/content/gdrive/MyDrive/CCL/sent_dev.json', "r", encoding="utf-8") as f:
    sent_dev = json.load(f)
with open('/content/gdrive/MyDrive/CCL/pie_dev.json', "r", encoding="utf-8") as f:
    pie_dev = json.load(f)
dict_dev = {k:e for k, e in zip(pie_dev,sent_dev)}
print(dict_dev)

{'monkey business': 'Architecturally magnificent, it is as clean as a whistle, and absolutely no monkey business goes on.', 'grass roots': 'From time to time, however, grass roots may turn brown or black and begin to rot due to excess water and the presence of a fungal disease.', 'marching orders': "The soldiers, their faces etched with determination, awaited the general's marching orders, their boots tapping impatiently on the dusty parade ground.", 'panda car': "We'd been waiting for ages before the panda car finally arrived and the black-and-white bears came out at the front of the growing crowd.", 'bread and butter': "It's Steve's bread and butter, and has been for some time.", 'chocolate teapot': 'Lindt has made a classy dark chocolate teapot a few years back to promote its range of chocolate-tea pairings.', "pig's ear": "Thus, when it wrote the new partition table it made a right pig's ear of it.", 'best man': 'All stories aside, Neil is a great mate and it made me extremely prou

In [ ]:
def classify_compound_type(compound, sentence):
    prompt = f"""
    Your task is to analyze the given compound word and sentence. Determine whether the compound word is used idiomatically or literally in the given context. Respond with either "idiomatic" or "literal" based on your analysis.
    Don't give any explanation! Just return "idiomatic" or "literal".

    Compound: {compound}
    Sentence: {sentence}

    Output: "idiomatic" or "literal"
    """
    try:
        response = openai.ChatCompletion.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "You are an idiom expert."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"


In [ ]:
def tag_saver(dict_pie_sent):
  dict_tag = dict()
  for k, e in dict_pie_sent.items():
      dict_tag[k] = classify_compound_type(k, e)
      print(f"Compound: {k}, Type: {dict_tag[k]}")
  return dict_tag

In [ ]:
com_tag_dict = dict()
for k, e in dict_idiom.items():
    compound = k
    sentence = e
    c_type = classify_compound_type(compound, sentence)
    com_tag_dict[compound] = c_type
    print(f"Compound: {compound}, Type: {c_type}")


Compound: elbow grease, Type: idiomatic
Compound: night owl, Type: idiomatic
Compound: heart of gold, Type: idiomatic
Compound: shrinking violet, Type: idiomatic
Compound: banana republic, Type: idiomatic
Compound: private eye, Type: idiomatic


KeyboardInterrupt: 

In [ ]:
for k, e in dict_literal.items():
    compound = k
    sentence = e
    c_type = classify_compound_type(compound, sentence)
    com_tag_dict[compound] = c_type
    print(f"Compound: {compound},  Type: {c_type}")


In [ ]:
def meaning_generator(compound, tag):
    prompt = f"""
    Your task is to give the idiomatic or literal meaning of the compound words, according to the given tag ('idiomatic' or 'literal').
    The meanings should not exceed 50 words. Don't give any explanation! Just return the meaning.

    Compound: {compound}
    Tag: {tag}

    Output:
    - Meaning: [brief explanation of the compound word, maximum 50 words]
    """
    try:
        response = openai.ChatCompletion.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "You are an idiom expert."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"



In [ ]:
def meaning_saver(dict_tag):
  dict_meaning = dict()
  for k, e in dict_tag.items():
      dict_meaning[k] = meaning_generator(k, e)[11: ]
      print(f"Compound: {k}, {dict_meaning[k]}")
  return dict_meaning

In [ ]:
dict_train_idiom = dict()
for i in pie_idiom:
  dict_train_idiom[i] = meaning_generator(i, 'idiomatic')[11: ]
  print(f"Compound: {i}, Meaning: {dict_train_idiom[i]}")


Compound: elbow grease, Meaning: Hard physical effort or manual labor required to complete a task.
Compound: night owl, Meaning: A person who stays up late at night or is active during the night.
Compound: heart of gold, Meaning: A kind, generous, and compassionate nature.
Compound: shrinking violet, Meaning: A shy or timid person who avoids attention.
Compound: banana republic, Meaning: A politically unstable country, typically reliant on limited agriculture, and often governed by a corrupt or authoritarian regime.
Compound: private eye, Meaning: A private investigator or detective hired to conduct surveillance or gather information.
Compound: pipe dream, Meaning: An unrealistic or impractical hope or plan.
Compound: rocket science, Meaning: Something very complex or difficult to understand.
Compound: nest egg, Meaning: Savings set aside for future use, especially for retirement.
Compound: bull market, Meaning: A period of rising stock prices, indicating investor confidence and econom

In [ ]:
with open('/content/gdrive/MyDrive/CCL/train_idiom_mean.json', "w", encoding="utf-8") as f:
    json.dump(dict_train_idiom, f)

In [ ]:
dict_train_literal = dict()
for i in pie_literal:
  dict_train_literal[i] = meaning_generator(i, 'literal')[11: ]
  print(f"Compound: {i}, Meaning: {dict_train_literal[i]}")

Compound: green fingers, Meaning: Fingers that are colored green, either due to paint, dye, or other substances.
Compound: ancient history, Meaning: The study or record of events, people, and societies from the distant past, typically before the Middle Ages.
Compound: devil's advocate, Meaning: A person who argues against a cause or position, not out of personal belief, but to test the validity of the argument or to provoke deeper discussion.
Compound: piece of cake, Meaning: A portion or slice of a baked dessert made from flour, sugar, eggs, and other ingredients.
Compound: brass ring, Meaning: A ring made of brass, a metal alloy consisting primarily of copper and zinc.
Compound: apples and oranges, Meaning: Two different types of fruits, specifically apples and oranges.
Compound: ghost town, Meaning: A deserted or abandoned town with few or no remaining inhabitants, often due to economic decline or natural disasters.
Compound: copy cat, Meaning: A cat that imitates or copies the acti

In [ ]:
with open('/content/gdrive/MyDrive/CCL/train_literal_mean.json', "w", encoding="utf-8") as f:
    json.dump(dict_train_literal, f)

In [ ]:
dev_tag = tag_saver(dict_dev)
print(dev_tag)


Compound: monkey business, Type: idiomatic
Compound: grass roots, Type: literal
Compound: marching orders, Type: literal
Compound: panda car, Type: literal
Compound: bread and butter, Type: idiomatic
Compound: chocolate teapot, Type: literal
Compound: pig's ear, Type: idiomatic
Compound: best man, Type: idiomatic
Compound: big cheese, Type: literal
Compound: eager beaver, Type: idiomatic
Compound: hair of the dog, Type: literal
Compound: thin ice, Type: idiomatic
Compound: snake in the grass, Type: idiomatic
Compound: flea market, Type: literal
Compound: big fish, Type: literal
{'monkey business': 'idiomatic', 'grass roots': 'literal', 'marching orders': 'literal', 'panda car': 'literal', 'bread and butter': 'idiomatic', 'chocolate teapot': 'literal', "pig's ear": 'idiomatic', 'best man': 'idiomatic', 'big cheese': 'literal', 'eager beaver': 'idiomatic', 'hair of the dog': 'literal', 'thin ice': 'idiomatic', 'snake in the grass': 'idiomatic', 'flea market': 'literal', 'big fish': 'lite

In [ ]:
dict_dev_mean = meaning_saver(dev_tag)
print(dict_dev_mean)

Compound: monkey business, Dishonest, mischievous, or deceitful behavior; foolish or silly actions.
Compound: grass roots, The roots of grass plants, typically found in the soil, which anchor the plant and absorb water and nutrients.
Compound: marching orders, Official instructions given to soldiers or a group to begin marching or moving in a specific direction.
Compound: panda car, A small, black-and-white police patrol car, resembling the colors of a panda, used for patrolling neighborhoods.
Compound: bread and butter, A person's primary source of income or main livelihood.
Compound: chocolate teapot, A teapot made entirely of chocolate, used for decorative or novelty purposes rather than functional use.
Compound: pig's ear, A mess or something done badly.
Compound: best man, The groom's chief attendant at a wedding, responsible for supporting him and often giving a speech.
Compound: big cheese, A large piece of cheese.
Compound: eager beaver, A person who is extremely enthusiastic, 

In [ ]:
with open('/content/gdrive/MyDrive/CCL/dev_mean.json', "w", encoding="utf-8") as f:
    json.dump(dict_dev_mean, f)